# Label

Run the models against fresh data to see how it goes.

In [18]:
import pickle
import pathlib
from urllib.parse import urlparse

In [19]:
import tldextract
import numpy as np
import pandas as pd

In [20]:
input_path = pathlib.Path("") / "input"

In [21]:
output_path = pathlib.Path("") / "output"

In [22]:
vectorizer = pickle.load(open(output_path / "vectorizer.pickle", "rb"))

In [23]:
model = pickle.load(open(output_path / "gnb.pickle", "rb"))

In [24]:
labeled_df = pd.read_csv(input_path / "labeled.csv", dtype={"is_story": int})

In [25]:
DOMAIN_BLACKLIST = (
    "google",
    "twitter",
    "facebook",
    "doubleclick",
    "instagram",
    "pinterest",
    "legacy",
)

In [26]:
SUBDOMAIN_BLACKLIST = (
    "careers",
    "mail",
    "account",
)

In [43]:
def label(url, text):
    path = urlparse(url).path
    tld = tldextract.extract(url)
    if tld.domain in DOMAIN_BLACKLIST:
        return False
    elif tld.subdomain in SUBDOMAIN_BLACKLIST:
        return False
    text_feature = vectorizer.transform([text]).toarray()
    path_feature = vectorizer.transform([path]).toarray()
    feature = np.concatenate([text_feature, path_feature], axis=1)
    return model.predict(feature)

In [44]:
label("http://www.latimes.com/whatever.html", "This is not a headline")

ValueError: X has 330 features, but GaussianNB is expecting 294 features as input.